<a href="https://colab.research.google.com/github/tanishq150802/ML7-RockPaperScissorGAME/blob/main/ML7_RockPaperScissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import random
import matplotlib.pyplot as plt
import cv2 as cv
from sklearn.metrics import f1_score

###Importing the dataset

https://www.tensorflow.org/datasets/catalog/rock_paper_scissors




In [ ]:
(ds_train, ds_valid, ds_test), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train[:80%]', 'train[80%:]', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteCBWNHJ/rock_paper_scissors-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteCBWNHJ/rock_paper_scissors-test.tfrecord


Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


###Pre-processing the data

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.Resizing(300,300, input_shape = (300,300,3)),                                       
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
])

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_valid = ds_valid.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_valid = ds_valid.cache()
ds_valid = ds_valid.shuffle(ds_info.splits['train'].num_examples)
ds_valid = ds_valid.batch(32)
ds_valid = ds_valid.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.cache()
ds_test = ds_test.shuffle(ds_info.splits['train'].num_examples)
ds_test = ds_test.batch(32)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

###Creating the model

In [ ]:
#Creating model

num_classes = 3
img_height = 300
img_width = 300
model = tf.keras.models.Sequential([
                    data_augmentation,
                    tf.keras.layers.Conv2D(32, 3, padding='same',  activation='relu',kernel_initializer='he_uniform',),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation='relu'),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Conv2D(128, 3, padding='same', kernel_initializer='he_uniform',activation='relu'),
                    tf.keras.layers.MaxPooling2D(2),
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(128, kernel_initializer='he_uniform',activation = 'relu'),
                    tf.keras.layers.Dense(3),
                    ])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 300, 300, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 64)       

###Compiling the model


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Training and testing the model

In [ ]:
model.fit(ds_train, epochs=3, validation_data=ds_valid)

Epoch 1/3
63/63 [==============================] - 43s 103ms/step - loss: 1.0873 - accuracy: 0.5799 - val_loss: 0.4289 - val_accuracy: 0.7639
Epoch 2/3
63/63 [==============================] - 5s 77ms/step - loss: 0.2719 - accuracy: 0.9018 - val_loss: 0.0900 - val_accuracy: 0.9762
Epoch 3/3
63/63 [==============================] - 5s 77ms/step - loss: 0.0911 - accuracy: 0.9752 - val_loss: 0.0197 - val_accuracy: 0.9940


In [ ]:
test_loss, test_acc = model.evaluate(ds_test)

print('\nTest accuracy:', test_acc)

12/12 [==============================] - 2s 64ms/step - loss: 0.1284 - accuracy: 0.9435

Test accuracy: 0.9435483813285828


In [ ]:
#Calculating F1 Score

test_image, test_label = list(), list()
ds = tfds.load('rock_paper_scissors', split='test')
for element in ds:
  test_image.append(element["image"])
  test_label.append(element["label"])
test_image = np.array(test_image)
test_label = np.array(test_label)

probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_image)

list_of_outcomes = list()
for pred in predictions:
  outcome = np.argmax(pred)
  list_of_outcomes.append(outcome)

list_of_truth = []
for a in range(len(test_label)):
  list_of_truth.append(test_label[a])


print('F1 score = ', f1_score(list_of_truth, list_of_outcomes, average = 'macro'))

F1 score =  0.9495933890440303


###Hyperparameter tuning

###Using the trained model for prediction

In [ ]:
#Getting image feed
#add the file path of the image for which you want to make the prediction.
path = "/content/Rock_paper_Seassors_1.png"
image = cv.imread(path)
# Loading the image

stretch_near = cv.resize(image,(300, 300),interpolation = cv.INTER_NEAREST)
stretch_near.shape
stretch_near = stretch_near[np.newaxis, :]
stretch_near.shape

In [ ]:
#Evaluating the move
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(stretch_near)
print(np.argmax(predictions), predictions[0])

if np.argmax(predictions) == 0:
  print("rock")
elif np.argmax(predictions) == 1:
  print("paper")
elif np.argmax(predictions) == 2:
  print("scissor")

###Collecting the live camera feed

###Creating the single/dual player game GUI using pygame

In [ ]:
!pip install pygame
import pygame as pg

###Addition: Predicting the next play of the user using another ML model

### References

Laurence Moroney, Rock, Paper, Scissors Dataset, Feb 2019  
http://laurencemoroney.com/rock-paper-scissors-dataset